In [373]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [374]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [375]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {x:entry for entry, x in enumerate(world_set)}

In [376]:
print(vocab)

{'e': 0, 'w': 1, 'i': 2, 'u': 3, 'a': 4, '.': 5, 'n': 6, 't': 7, 'y': 8, ',': 9, 'o': 10, 'l': 11, 'r': 12, 'd': 13, 'm': 14, 'b': 15, 'f': 16, 'c': 17, 'B': 18, ' ': 19, 's': 20, 'p': 21, 'g': 22, 'h': 23, "'": 24, 'k': 25}


In [377]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [378]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [379]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:sequence_length+i]
  y_str = sentence[i+1:sequence_length+i+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [380]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[18, 12, 2, 17, 25, 19, 1, 4, 11, 11]
[12, 2, 17, 25, 19, 1, 4, 11, 11, 20]


In [381]:
np.max(x_data)

25

In [382]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(np.max(x_data)+1)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [383]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [384]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [385]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([12,  2, 17, 25, 19,  1,  4, 11, 11, 20])


In [386]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(input_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [387]:
net = Net(vocab_size, hidden_size, 2)

In [388]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [389]:
X.shape

torch.Size([188, 10, 26])

In [390]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [391]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

innininnnnnnnninniininnnnnnnninnnnnnnnnnnninnininnnnnnnnnnnnnninninnninninnnnnnnninnnnnnininnnnniinnnnnnnnnnnnninnnnininnnnnnnnnnnnninninninnninninnnnnnnninniininnnnnnnninnnnnninninnnninnnnninnnnnn
ineieeeeeeeeeeeeeeeeeeeeeeeeeeeeeieeeeeeeeeeeieeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
iee e eeee eeeeeeeeeeeeeeeeeeeeee eeeeeeee ee e ee eeeeee eeeeeeeeeeeeee eeee eeeeeeeeeeeeeeee eeeeee eee eeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeee eeeeeeeeeeeee ee eee eeeeeeeeeeeeeeeeee eeeee 
 e                                                                                                                                                                                                   
                                                                                                                                                                                                     
          

In [392]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinksm"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [393]:
 # 베이스라인 기반으로 한국어 학습 - 동일한 특정 단어(공장장, 철창살)가 반복되는 문장은 어떻게 학습할 것인가?
 # 소규모 데이터이므로 토크나이저를 쓰는 대신 사전에 문장 부호를 제거하여 입력
 # 띄어쓰기 단위로 문장 구분해 학습

sentence = ("간장 공장 공장장은 강 공장장이고 "
            "된장 공장 공장장은 장 공장장이고 "
            "강된장 공장 공장장은 공 공장장이다 "
            "경찰청 철창살은 외철창살이고 "
            "검찰청 철창살은 쌍철창살이다 ")

word_split = sentence.split()

num=0
for i in word_split :
  word_split[num] = i + ' '
  num += 1

word_split 

['간장 ',
 '공장 ',
 '공장장은 ',
 '강 ',
 '공장장이고 ',
 '된장 ',
 '공장 ',
 '공장장은 ',
 '장 ',
 '공장장이고 ',
 '강된장 ',
 '공장 ',
 '공장장은 ',
 '공 ',
 '공장장이다 ',
 '경찰청 ',
 '철창살은 ',
 '외철창살이고 ',
 '검찰청 ',
 '철창살은 ',
 '쌍철창살이다 ']

In [394]:
## 각 단어에 정수 인코딩
vocab = {x:entry for entry, x in enumerate(set(word_split))}
print(vocab)

{'장 ': 0, '공장장이고 ': 1, '강된장 ': 2, '공 ': 3, '철창살은 ': 4, '강 ': 5, '공장장은 ': 6, '공장장이다 ': 7, '외철창살이고 ': 8, '쌍철창살이다 ': 9, '공장 ': 10, '경찰청 ': 11, '간장 ': 12, '된장 ': 13, '검찰청 ': 14}


In [395]:
vocab_size = len(vocab)
print('단어어 집합 크기 : {}'.format(vocab_size))

단어어 집합 크기 : 15


In [396]:
# 문장이 대개 5개 단어로 이루어지므로 시퀀스 길이도 5로 설정
# 민감성 고려해 학습률 조정
hidden_size = vocab_size 
sequence_length = 5
learning_rate = 0.005
x_data = []
y_data = []

for i in range(0, len(word_split) - sequence_length):
  x_str = word_split[i:sequence_length+i]
  y_str = word_split[i+1:sequence_length+i+1]
  print(i, x_str, "->", y_str)
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['간장 ', '공장 ', '공장장은 ', '강 ', '공장장이고 '] -> ['공장 ', '공장장은 ', '강 ', '공장장이고 ', '된장 ']
1 ['공장 ', '공장장은 ', '강 ', '공장장이고 ', '된장 '] -> ['공장장은 ', '강 ', '공장장이고 ', '된장 ', '공장 ']
2 ['공장장은 ', '강 ', '공장장이고 ', '된장 ', '공장 '] -> ['강 ', '공장장이고 ', '된장 ', '공장 ', '공장장은 ']
3 ['강 ', '공장장이고 ', '된장 ', '공장 ', '공장장은 '] -> ['공장장이고 ', '된장 ', '공장 ', '공장장은 ', '장 ']
4 ['공장장이고 ', '된장 ', '공장 ', '공장장은 ', '장 '] -> ['된장 ', '공장 ', '공장장은 ', '장 ', '공장장이고 ']
5 ['된장 ', '공장 ', '공장장은 ', '장 ', '공장장이고 '] -> ['공장 ', '공장장은 ', '장 ', '공장장이고 ', '강된장 ']
6 ['공장 ', '공장장은 ', '장 ', '공장장이고 ', '강된장 '] -> ['공장장은 ', '장 ', '공장장이고 ', '강된장 ', '공장 ']
7 ['공장장은 ', '장 ', '공장장이고 ', '강된장 ', '공장 '] -> ['장 ', '공장장이고 ', '강된장 ', '공장 ', '공장장은 ']
8 ['장 ', '공장장이고 ', '강된장 ', '공장 ', '공장장은 '] -> ['공장장이고 ', '강된장 ', '공장 ', '공장장은 ', '공 ']
9 ['공장장이고 ', '강된장 ', '공장 ', '공장장은 ', '공 '] -> ['강된장 ', '공장 ', '공장장은 ', '공 ', '공장장이다 ']
10 ['강된장 ', '공장 ', '공장장은 ', '공 ', '공장장이다 '] -> ['공장 ', '공장장은 ', '공 ', '공장장이다 ', '경찰청 ']
11 ['공장 ', '공장장은 ', '공 ', '공장장이다 ', '경찰청 '] -> ['공장장은

In [397]:
x_data

[[12, 10, 6, 5, 1],
 [10, 6, 5, 1, 13],
 [6, 5, 1, 13, 10],
 [5, 1, 13, 10, 6],
 [1, 13, 10, 6, 0],
 [13, 10, 6, 0, 1],
 [10, 6, 0, 1, 2],
 [6, 0, 1, 2, 10],
 [0, 1, 2, 10, 6],
 [1, 2, 10, 6, 3],
 [2, 10, 6, 3, 7],
 [10, 6, 3, 7, 11],
 [6, 3, 7, 11, 4],
 [3, 7, 11, 4, 8],
 [7, 11, 4, 8, 14],
 [11, 4, 8, 14, 4]]

In [398]:
x_one_hot = np.eye(np.max(x_data)+1)[np.array(x_data)]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([16, 5, 15])
레이블의 크기 : torch.Size([16, 5])


In [399]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(input_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x
net = Net(vocab_size, hidden_size, 5)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)
print(outputs.shape)

torch.Size([16, 5, 15])


In [400]:
# Train 결과 '강된장 -> 공장 -> 강 -> 공장 -> 공장장은 -> 강된장 -> 공장...'의 무한 루프에 빠지는 것으로 확인됨...
# 반면 '공장장이다 -> 경찰청'을 통해 다음 문장으로 넘어가는 순서를 저평가해 학습한 것으로 추정
# 이처럼 단순한 구조의 RNN에서는 데이터셋 자체의 크기가 작을 때, 특정 값이 반복되면 해당 값 주위의 연결을 과다 학습할 수 있는 것으로 이해하면 될까요?
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: 
            predict_str += ''.join([word_split[t] for t in result])
        else: 
            predict_str += word_split[result[-1]]

    print(predict_str)

강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 
강 강된장 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 
강 강된장 강 강된장 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 강 
강 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 강된장 공장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 강된장 공장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 공장 공장 강된장 강된장 강된장 공장 강된장 강된장 강된장 강된장 공장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 강된장 
강된장 공장 공장 강된장 공장 강된장 공장 강된장 강된장 공장 강된장 공장 강된장 강된장 공장 강된장 공장 강된장 강된장 공장 
강된장 공장 공장 강된장 공장 강된장 공장 공장장이고 강된장 공장 강된장 공장 공장장이고 강된장 공장 공장장이고 공장 공장 강된장 공장 
강된장 공장 공장장이고 강된장 공장 강된장 공장 공장장이고 강된장 공장 공장장이고 공장 공장장이고 강된장 공장 공장장이고 공장 공장장이고 강된장 공장 
강된장 공장 공장장이고 강된장 공장 공장장이고 공장 공장장이고 강된장 공장 공장장이고 공장 공장장이고 강된장 공장 공장장이고 공장 공장장이고 강

In [401]:
 # 반대로 반복되는 단어가 없는 소규모 데이터셋을 학습한 경우는 어떻게 될까?

sentence = ("죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를 "
            "잎새에 이는 바람에도 나는 괴로워했다 "
            "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지 "
            "그리고 나한테 주어진 길을 걸어가야겠다 "
            "오늘 밤에도 별이 바람에 스치운다 ")

word_split = sentence.split()

num=0
for i in word_split :
  word_split[num] = i + ' '
  num += 1

vocab = {x:entry for entry, x in enumerate(set(word_split))}
vocab_size = len(vocab)
print("단어의 수: ", len(word_split))
print('단어 집합 크기 : {}'.format(vocab_size))
print(vocab)

hidden_size = vocab_size 
sequence_length = 5
learning_rate = 0.005
x_data = []
y_data = []

for i in range(0, len(word_split) - sequence_length):
  x_str = word_split[i:sequence_length+i]
  y_str = word_split[i+1:sequence_length+i+1]
  print(i, x_str, "->", y_str)
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

x_one_hot = np.eye(np.max(x_data)+1)[np.array(x_data)]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(input_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x
net = Net(vocab_size, hidden_size, 5)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)
print(outputs.shape)

단어의 수:  31
단어 집합 크기 : 31
{'나는 ': 0, '바람에 ': 1, '이는 ': 2, '사랑해야지 ': 3, '가는 ': 4, '주어진 ': 5, '마음으로 ': 6, '밤에도 ': 7, '별이 ': 8, '것을 ': 9, '길을 ': 10, '부끄럼이 ': 11, '걸어가야겠다 ': 12, '노래하는 ': 13, '모든 ': 14, '죽는 ': 15, '오늘 ': 16, '점 ': 17, '없기를 ': 18, '날까지 ': 19, '그리고 ': 20, '죽어 ': 21, '한 ': 22, '하늘을 ': 23, '나한테 ': 24, '바람에도 ': 25, '스치운다 ': 26, '우러러 ': 27, '잎새에 ': 28, '괴로워했다 ': 29, '별을 ': 30}
0 ['죽는 ', '날까지 ', '하늘을 ', '우러러 ', '한 '] -> ['날까지 ', '하늘을 ', '우러러 ', '한 ', '점 ']
1 ['날까지 ', '하늘을 ', '우러러 ', '한 ', '점 '] -> ['하늘을 ', '우러러 ', '한 ', '점 ', '부끄럼이 ']
2 ['하늘을 ', '우러러 ', '한 ', '점 ', '부끄럼이 '] -> ['우러러 ', '한 ', '점 ', '부끄럼이 ', '없기를 ']
3 ['우러러 ', '한 ', '점 ', '부끄럼이 ', '없기를 '] -> ['한 ', '점 ', '부끄럼이 ', '없기를 ', '잎새에 ']
4 ['한 ', '점 ', '부끄럼이 ', '없기를 ', '잎새에 '] -> ['점 ', '부끄럼이 ', '없기를 ', '잎새에 ', '이는 ']
5 ['점 ', '부끄럼이 ', '없기를 ', '잎새에 ', '이는 '] -> ['부끄럼이 ', '없기를 ', '잎새에 ', '이는 ', '바람에도 ']
6 ['부끄럼이 ', '없기를 ', '잎새에 ', '이는 ', '바람에도 '] -> ['없기를 ', '잎새에 ', '이는 ', '바람에도 ', '나는 ']
7 ['없기를 ', '잎새에 ', '이는 ', '바람에도 ', '나는

In [402]:
# 무한 루프에는 빠지지 않으나, 문장 전체적인 흐름이 매끄럽지 못함... 간헐적으로 2~3개 단어 사이만 적절하게 연결되는 것이 확인됨
# 애초에 모든 단어가 한번씩만 나오기 때문에 새로우면서 자연스러운 문장을 기대할 수 없음

for i in range(50):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: 
            predict_str += ''.join([word_split[t] for t in result])
        else: 
            predict_str += word_split[result[-1]]

    print(predict_str)

부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 
부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 부끄럼이 
부끄럼이 부끄럼이 바람에도 괴로워했다 괴로워했다 바람에도 이는 괴로워했다 부끄럼이 이는 바람에도 바람에도 부끄럼이 바람에도 부끄럼이 괴로워했다 바람에도 바람에도 이는 바람에도 바람에도 이는 괴로워했다 바람에도 바람에도 이는 바람에도 괴로워했다 바람에도 이는 
부끄럼이 이는 바람에도 괴로워했다 괴로워했다 괴로워했다 이는 괴로워했다 괴로워했다 이는 괴로워했다 괴로워했다 괴로워했다 괴로워했다 부끄럼이 괴로워했다 괴로워했다 괴로워했다 이는 괴로워했다 바람에도 이는 괴로워했다 괴로워했다 괴로워했다 괴로워했다 괴로워했다 괴로워했다 괴로워했다 이는 
부끄럼이 이는 바람에도 괴로워했다 괴로워했다 괴로워했다 부끄럼이 괴로워했다 하늘을 이는 하늘을 괴로워했다 부끄럼이 부끄럼이 부끄럼이 괴로워했다 괴로워했다 부끄럼이 이는 괴로워했다 부끄럼이 부끄럼이 괴로워했다 괴로워했다 괴로워했다 괴로워했다 괴로워했다 괴로워했다 부끄럼이 이는 
부끄럼이 이는 바람에도 사랑해야지 부끄럼이 부끄럼이 부끄럼이 괴로워했다 하늘을 이는 하늘을 부끄럼이 부끄럼이 부끄럼이 부끄럼이 노래하는 하늘을 부끄럼이 이는 괴로워했다 사랑해야지 부끄럼이 괴로워했다 사랑해야지 부끄럼이 이는 괴로워했다 부끄럼이 하늘을 부끄럼이 
부끄럼이 노래하는 노래하는 사랑해야지 부끄럼이 부끄럼이 부끄럼이 노래하는 하늘을 이는 하늘을 가는 부끄럼이 사랑해야지 부끄럼이 노래하는 그리고 부끄럼이 이는 없기를 사